In [7]:
import pandas as pd
import re

In [8]:
data_path = "../data/naruto-transcript-ep1.csv"
transcript = pd.read_csv(data_path)

In [9]:
transcript

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah
...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...
159,Iruka,(Opens his eyes) Huh?
160,Naruto,"Iruka Sensei! (Naruto jumps at him, hugging h..."
161,Iruka,(Laughing) Ah! That hurts!


In [10]:
def remove_parathesis(text):
    result = re.sub(r'\(.*?\)','',text)
    return result


In [11]:
transcript['line'] = transcript['line'].apply(remove_parathesis)

In [13]:
transcript.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [14]:
transcript['num_of_words'] = transcript['line'].str.strip().str.split(" ")
transcript["num_of_words"] = transcript['num_of_words'].apply(lambda x:len(x))


In [15]:
transcript.head()

,name,line,num_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [19]:
transcript['naruto_response_flag'] = 0
transcript.loc[(transcript['name'] == "Naruto")&(transcript["num_of_words"]>1),'naruto_response_flag'] = 1

In [21]:
transcript

,name,line,num_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,1
161,Iruka,Ah! That hurts!,3,0


In [22]:
indexes_to_take = list(transcript[(transcript["naruto_response_flag"]==1)].index)

In [23]:
system_prompt =  """ Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pattern\n """

prompts =[]
for ind in indexes_to_take:
    prompt = system_prompt
    
    prompt += transcript.iloc[ind-1]['line']
    prompt += '\n'
    prompt += transcript.iloc[ind]['line']

    prompts.append(prompt) 

In [24]:
prompts

[' Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pattern\n   Naruto. This is only the beginning. The road gets tougher now that you’re a Ninja. But if I told you that, I guess it would ruin the moment. So I’ll tell you later. Over 500 bowls of Ramen.\n  Give it up.  You’re just bent, because you didn’t have the guts to do what I do. Do ya!? Losers! Wanabees! You’ll never catch me! ',
 ' Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pattern\n   Oh yeah, Naruto!?\n Where’d you come from, Iruka Sensei!? What are you doing here?',
 ' Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pattern\n  We always pay for your screw ups.\n Like I care.',
 ' Your are Naruto from the anime "Naruto". Your responses should reflect his personality and speech pattern\n  Huh!? \n Gotcha! That’s my Sexy Jutsu!',
 ' Your are Naruto from the anime "Naruto". You

In [26]:
df  = pd.DataFrame({'prompt':prompts})
df.head()

,prompt
0,"Your are Naruto from the anime ""Naruto"". Your..."
1,"Your are Naruto from the anime ""Naruto"". Your..."
2,"Your are Naruto from the anime ""Naruto"". Your..."
3,"Your are Naruto from the anime ""Naruto"". Your..."
4,"Your are Naruto from the anime ""Naruto"". Your..."


In [27]:
from datasets import Dataset

In [28]:
dataset = Dataset.from_pandas(df)